<div align=center>
		
<p></p>
<p></p>
<font size=5>
In the Name of God
<font/>
<p></p>
 <br/>
    <br/>
    <br/>
<font color=#FF7500>
Sharif University of Technology - Departmenet of Computer Engineering
</font>
<p></p>
<font color=blue>
Linear Algebra 
<br>
Prof. Hamid R. Rabiee
<br>
Dr. Maryam Ramezani
</font>
<br/>
<br/>
Fall 2022

</div>

<hr/>
		<div align=center>
		    <font color=red size=6>
			    <br />
Project - Q1 <br>
Watermark Images using SVD

(35% of the overall project grade)
            	<br/>
			</font>
<font size=4>
			<br/><br/>
<font color=red>
Please run all the cells.
     </font>
</font>
<br>
<font size=3 color=grey>
</font>
    </div>

Name: Amir Hossein Naqdalie
<br>
Student Number: 400105296

# Watermark

A watermark is an image, overlay, or text that's placed over a digital asset. Usually, the image will be monochromatic and transparent, to enable others to still view the asset. You can read about the importance of the watermark in this [link](https://www.mediavalet.com/blog/watermarks-are-important/#).

# Add Watermark

## Imports

In [81]:
import numpy as np
import cv2
# from google.colab.patches import cv2_imshow
from PIL import Image

## Load & Show Images
If you are using colab, you can upload the files by running the following cell.

In [82]:
# from google.colab import files
# uploaded = files.upload()

In the following cell, load the original image and the logo from the folder add_watermark and plot them using the ```cv2``` library. 

Note: both the image and the logo should be gray-scale.

In [83]:
original_img = cv2.imread('add_watermark/dog.png', 0)
watermark_img = cv2.imread('add_watermark/logo.png', 0)

cv2.imshow("Dog", original_img)
cv2.waitKey(2000)
cv2.destroyAllWindows()

cv2.imshow("Logo", watermark_img)
cv2.waitKey(2000)
cv2.destroyAllWindows()

## Implementation [50 points]

In this part you should add watermark to the original image. You should implement the following algorithm.

1. Perform SVD on the original image (A matrix) and form the result as $USV^T$.

2. Add watermark (W matrix), muliplied by a coefficine K, to the SVs of the original matrix. (S matrix in the previous step)

3. Now, perform SVD again on matrix D.

4. Replace the matrix S in the first step, with the SVs of the previous step (the middle matrix in step 3) to get the watermarked image.



In [84]:
def encode_watermark(original_img, watermark_img, watermarked_coeff=0.7):
    U, S, VH = np.linalg.svd(original_img, full_matrices=False)
    
    D = watermarked_coeff * watermark_img + np.diag(S)
    
    S_D = np.linalg.svd(D, full_matrices=False, compute_uv=False)

    watermarked_image = U @ np.diag(S_D) @ VH
    watermarked_image = watermarked_image.astype(np.uint8)
    return watermarked_image

Display the watermarked image. 

In [85]:
watermarked_coeff = [0, 0.00001, 0.1, 0.7, 2, 10, 50, 100, 200]

for coeff in watermarked_coeff:
    watermarked_img = encode_watermark(original_img, watermark_img, coeff)
    cv2.imshow("Watermarked " + str(coeff), watermarked_img)
    cv2.waitKey(2000)
    cv2.destroyAllWindows()

Explain about the place you added the logo. How is the watermarked image visually different from the original image? [5 points]

Answer: Since we apply changes to singular values matrix of the original image, that's where the logo is added in the new image. Watermark image will somehow be a transparent texture for the original image.(We can say that rate of transparency and how much watermark affects the watermarked image depends on the value of watermarking coefficient)

# Watermark Extraction [45 points]

## Load Files

In [86]:
# from google.colab import files
# uploaded = files.upload()

In [87]:
watermarked_u = np.load('extract_watermark/watermarked_u.npy')
watermarked_vh = np.load('extract_watermark/watermarked_vh.npy')
S = np.load('extract_watermark/S.npy')
watermarked_img = cv2.imread('extract_watermark/watermarked_image.jpg', 0)

## Implementation 

The task in this part is to extract the watermarked image from a distorted one ($A^*_w$ matrix) and write the sentence that is written in the watermarked image. The matrix of singular values (S) from the original image, $U_w$, $V^T_w$, and the watermarked image are given to you.

1. Perform SVD on the distorted watermarked image ($A^*_w$ matrix) in the form of $U^* S^*_w V^{*T}$.

2. Compute the matrix that includes the watermark by $D^* = U_w S^*_w V^T_w$.

3. Compute the corrupted watermark by subtracting the matrix $S$ from $D^*$ and dividing the result by the coefficient of the watermark, $K$.


In [88]:
def get_watermark(watermarked_img, watermarked_u, S, watermarked_vh, watermarked_coeff):
    U, S, VH = np.linalg.svd(watermarked_img, full_matrices=False)
    D = watermarked_u @ np.diag(S) @ watermarked_vh
    watermark = (D - S) / watermarked_coeff
    return watermark

Get the watermark logo and display it. Note that in this section we do not know the watermark coefficient. You have to run the previous cell's function for at least 5 different coefficients and compare the results.   

In [89]:
watermarked_coeff = [0.00001, 0.1, 0.7, 1, 2, 10, 50, 100, 200, 500, 1000]

for coeff in watermarked_coeff:
    watermark_img = get_watermark(watermarked_img, watermarked_u, S, watermarked_vh, coeff)
    cv2.imshow("Watermark " + str(coeff), watermark_img)
    cv2.waitKey(2000)
    cv2.destroyAllWindows()

Explain the effect of different coefficient values on the quality of the displayed logo. [5 points]

Answer: For different coefficients we recieve different multiplicants of the original watermark image. If we choose coefficient less than the actual coefficient we recieve a * watermark where a > 1. And if we choose coefficient bigger than the actual one we recieve a * watermark where a < 1.(And clearly for chosen_coefficient=actual_coefficient we have recieved_watermark=actual_watermark) So for values which are closer to the original value we will have higher qualities, and the more we get further from it the worse the quality gets.